In [1]:
import torch
from torch.utils.data import DataLoader
from object_detection_data import ObjectDetectionDataset, Encoder, TransformedObjectDetectionDataset
import torch.nn as nn
from object_detection_models import MultiClassJetNet, LightningMultiClassJetNet
from pytorch_lightning.callbacks import RichProgressBar
import pytorch_lightning as pl
torch.manual_seed(2)


# TODO
- Select the correct scalings for the default maps
- Select the correct initial learning rate
- Add learning rate schedule
- Log additional metrics
- Benchmark training loop
- Show Network predictions for example images
- Add data augmentation


# Open questions
- What other kinds of data augmentation can I use?
- How do I parametrize the scalings of the default boxes?
- Do I need a learning rate schedule?
- Can I speed up the training process?
  - Make use of torch compile?
- Am I missing something?

In [4]:
default_scalings = [torch.tensor(
    [0.25, 0.25]), torch.tensor(
    [0.125, 0.125]), torch.tensor([0.125 / 2, 0.125 / 2]), torch.tensor([0.125 / 4, 0.125 / 4])]
feature_map_size = (8, 10)
num_classes = 4
encoder = Encoder(default_scalings, feature_map_size, num_classes)
data_path = 'SPLObjDetectDatasetV2/train'
transformed_train_data = TransformedObjectDetectionDataset(
    'SPLObjDetectDatasetV2/train', encoder)
transformed_val_data = TransformedObjectDetectionDataset(
    'SPLObjDetectDatasetV2/val', encoder)
train_loader = DataLoader(transformed_train_data, batch_size=8,
                          shuffle=True, num_workers=8)
val_loader = DataLoader(transformed_val_data, batch_size=8,
                        shuffle=False, num_workers=8)


In [5]:
pl_model = LightningMultiClassJetNet(num_classes, len(default_scalings))
trainer = pl.Trainer(limit_predict_batches=100,
                     max_epochs=10, callbacks=RichProgressBar())
trainer.fit(model=pl_model, train_dataloaders=train_loader, val_dataloaders=val_loader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


┏━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name  ┃ Type            ┃ Params ┃
┡━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model │ OptimizedModule │ 27.1 K │
└───┴───────┴─────────────────┴────────┘

Trainable params: 27.1 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 27.1 K                                                                                               
Total estimated model params size (MB): 0

Output()